#### Load required packages 

In [ ]:
import os
import glob
import pickle 
import pandas as pd
import numpy as np

In [ ]:
from dask.diagnostics import ProgressBar 
from arboreto.utils import load_tf_names
from arboreto.algo import grnboost2
from pyscenic.rnkdb import FeatherRankingDatabase as RankingDatabase
from pyscenic.utils import modules_from_adjacencies, load_motifs 
from pyscenic.prune import prune2df, df2regulons
from pyscenic.aucell import aucell 
import json
import time

#### Define variables 

In [ ]:
# Directory to hold results from pySCENIC
output_folder="pySCENIC/outs"
regulons_fname=os.path.join(output_folder, "regulons.p")
motifs_fname=os.path.join(output_folder, "motifs.csv")
# Directory to read in database
database_folder="scenic_database"
database_glob=os.path.join(database_folder, "hg38__*.feather")
motif_annotations_fname=os.path.join(database_folder, "motifs-v9-nr.hgnc-m0.001-o0.0.tbl")
hm_tfs_fname=os.path.join(database_folder, "hs_hgnc_curated_tfs.txt")
# Directory to read in inputs for pySCENIC, results from 08_exportScenic
resources_folder="output"
sc_expr_fname="".join([resources_folder, "/Competition_forScenic.csv"])

### 1. Explore imported variables 

In [ ]:
ex_matrix=pd.read_csv(sc_expr_fname, header=0, index_col=0).T
ex_matrix.head()

In [ ]:
tf_names=load_tf_names(hm_tfs_fname)

In [ ]:
ex_matrix.shape

In [ ]:
db_fnames=glob.glob(database_glob)
def name(fname): 
    return os.path.basename(fname).split(".")[0]
dbs=[RankingDatabase(fname=fname, name=name(fname)) for fname in db_fnames]
dbs

In [ ]:
# Modules are loaded from pickle
with open("modules.txt", "rb") as fp: 
    modules=pickle.load(fp)

### 2. Prune modules for targets whose DNA footprints to specific TFs were annotated

In [ ]:
# Begining time is 
t0=time.time()
df=prune2df(dbs, modules, motif_annotations_fname)

In [ ]:
# Ending time is 
t1=time.time()
# The total time it takes to prune modules in minutes
(t1-t0)/60

In [ ]:
# Save df 
with open("pruned.txt", "wb") as fp: 
    pickle.dump(df, fp)

In [ ]:
# Create regulons from this table of enriched motifs
regulons=df2regulons(df)

In [ ]:
# Save enriched motifs and the discovered regulons to disk 
df.to_csv(motifs_fname)
with open(regulons_fname, "wb") as f: 
    pickle.dump(regulons, f)

### 3. Export a text file of TF-target, equivalent to the RcisTarget output of R, where regulon-transcription-factor pairs are unique

In [ ]:
# Write a dictionary of {TF: Target} pairs 
rdictTarget={}
# Write a dictionary of {TF: Weight} pairs. The weight will be in the same order of Target
rdictWeight={}
for reg in regulons:
  targets = [ target for target in reg.gene2weight ]
  rdictTarget[reg.name] = targets
  weights = [ reg.gene2weight[target] for target in reg.gene2weight]
  rdictWeight[reg.name] = weights
print(rdictTarget.keys())
print(rdictWeight.keys())

In [ ]:
# Write to JSON for import to R
rjson = json.dumps(rdictTarget)
f = open("regulonsTarget.json","w")
f.write(rjson)
f.close()

rjson = json.dumps(rdictWeight)
f = open("regulonsWeight.json","w")
f.write(rjson)
f.close()

### 4. Export auc.txt matrix for plotting with R

In [ ]:
# Discover regulons specific to subset of cells
auc_mtx=aucell(ex_matrix, regulons, num_workers=8)

In [ ]:
##### Export auc_matrix for plotting in R
auc_mtx.to_csv("auc.txt")

# END OF SCRIPT